# Working with SAS

* **Difficulty level**: easy
* **Time need to lean**: 10 minutes or less
* **Key points**:
  * You can only exchanged SAS datasets as dataframes with magics `%get` and `%put`.
  * SAS statements can start with `%`, you will have to prefix them with a new line to avoid being handled as SoS magics.
  * If the dataset if not in the `WORK` library, you will have to pass the SAS dataset with name `libname.dataset`, and the dataset will be obtained as a dataframe with name `libname_dataset`.

## Installation

To use SAS with SoS, you will need to setup [`sos-kernel`](https://github.com/sassoftware/sas_kernel) and [`saspy`](https://github.com/sassoftware/saspy). The trickiest part is to the creation of a `sascfg_personal.py` file that specifies how to connect to the SAS server from the Jupyter server.

After you set up a `sos-kernel`, verified that the `sas` kernel appears in the list of `jupyter kernelspec list`, you should start a Jupyter notebook with a `SAS` kernel and verify if you can execute `SAS` codes from the Jupyter notebook. Then you can install `sos-sas` with command

```
pip install sos-sas
```
and then select `SAS` as the language for SAS cells.

**NOTE**: SoS `%get` datasets from SAS by importing data from the SAS data files (`.sas7bdat` files) directly. This is not possible if the SAS server is on a remote windows server that is connected by the `sas-kernel` via the IOM method. In another word, getting datasets from SAS only works if the SAS and Jupyter servers share the same server or file system, or connected via `SSH`.

## SAS `%` statements

The SAS language has syntaxes that look like Jupyter magics. To avoid SoS treating them as SoS magics, you will have to prefix the statement with a new line. For example, to execute a SAS `%put` statement as the first statement in a cell, you will have to enter it as the second line.

In [1]:
   
%put "this is SAS magic"

Using SAS Config named: ssh
SAS Connection established. Subprocess id is 43612



## SAS HTML Output

The `sas-kernel` generates output in HTML format, with embedded CSS (style sheet). Although the CSS is designed only for SAS outputs, they can change the look and feel of the entire SoS Notebook, even the HTML report generated from SoS Notebooks with SAS outputs. For example, the page you are reading looks a bit funny with elements (e.g. table of contents) look different from other pages, because of these embedded CSS.

## Data exachange between SAS and other kernels

A SAS dataset is roughly equivalent to Python's DataFrame (Pandas) and R's dataframe, and this is the only data type that can be exchanged between SAS and other languages.

### Sending dataframes to SAS

Data frames in languages such as Python and R can be send to SAS with the usual `%get` or `%put` magic. For example, you can `%get` a dataset from R as follows:

In [2]:
%get mtcars --from R
PROC PRINT;
RUN;

Obs,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
1,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
2,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
3,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1
4,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
5,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2
6,18.1,6,225.0,105,2.76,3.460,20.22,1,0,3,1
7,14.3,8,360.0,245,3.21,3.570,15.84,0,0,3,4
8,24.4,4,146.7,62,3.69,3.190,20.00,1,0,4,2
9,22.8,4,140.8,95,3.92,3.150,22.90,1,0,4,2
10,19.2,6,167.6,123,3.92,3.440,18.30,1,0,4,4


As you can see, SoS basically creates a dataset `MTCARS` in the `WORK` library. There is no way for you to specify name of a library so you will have to copy the datasets to a library in SAS if you would like to save the dataset permanently.

In [3]:
ods output Members=Members;
proc datasets library=work memtype=data;
run;
quit;

The following example shows the passing of a Python Pandas dataframe to SAS and plot it there. The data is of medium size of 100k rows, but should take just a few seconds to be passed to SAS.

In [4]:
%put df --to SAS
import pandas as pd
import numpy as np

# create a sample dataframe with 100,000 rows
x = np.linspace(0, 7, num=100000)
df = pd.DataFrame({
    'x': x,
    'y': np.sin(x)
})

In [5]:
ods html5;
ods graphics /width=500 height=400;
PROC GPLOT data=DF;
   title "The Sin Curve";
   plot y*x;
RUN;

### Passing SAS Datasets to other languages

If you have a SAS dataset, for example, a `MYCLASS` dataset as follows

In [6]:
DATA MYCLASS;
     INPUT NAME $ 1-8 SEX $ 10 AGE 12-13 HEIGHT 15-16 WEIGHT 18-22;
CARDS;
JOHN     M 12 59 99.5
JAMES    M 12 57 83.0
ALFRED   M 14 69 112.5
ALICE    F 13 56 84.0
PROC PRINT;
RUN;

Obs,NAME,SEX,AGE,HEIGHT,WEIGHT
1,JOHN,M,12,59,99.5
2,JAMES,M,12,57,83.0
3,ALFRED,M,14,69,112.5
4,ALICE,F,13,56,84.0


You can transfer the dataset to Python or R and analyze there. Note that SAS dataset is case insensitive so you can get either `myclass` or `MYCLASS`, but the variable name will be case sensitive in Python.

In [7]:
%get myclass --from SAS
%preview -n myclass

>>> myclass:

,NAME,SEX,AGE,HEIGHT,WEIGHT
0,JOHN,M,12.0,59.0,99.5
1,JAMES,M,12.0,57.0,83.0
2,ALFRED,M,14.0,69.0,112.5
3,ALICE,F,13.0,56.0,84.0


If the data is not in the `WORK` library, you will have to specify library name in the `%get` (or `%put` magic). For example, you can get the `sashelp.air` dataset from SAS with name `sashelp.air`. However, because this is not a valid name for a single variable, the dataset is obtained as `sashelp_air`.

In [8]:
%preview sashelp_air
%get sashelp.air --from SAS

>>> sashelp_air:

,DATE,AIR
0,1949-01-01,112.0
1,1949-02-01,118.0
2,1949-03-01,132.0
3,1949-04-01,129.0
4,1949-05-01,121.0
5,1949-06-01,135.0
6,1949-07-01,148.0
7,1949-08-01,148.0
8,1949-09-01,136.0
9,1949-10-01,119.0
